# 🔥 Fire Alarm System - Reliability Validation Layer 테스트
## Time-Series Queue → Voting Logic → Action Trigger 완전 동작 확인용

In [1]:
from collections import deque
import numpy as np
from datetime import datetime
import time
import random
import os
import cv2
from IPython.display import clear_output, display, Image

print("✅ 모든 라이브러리 로드 완료")

✅ 모든 라이브러리 로드 완료


# 초기 버전

In [5]:
class SmartTemporalLogic:
    """E-gle Eye Reliability Validation Layer
    VLM JSON 결과를 받아 Time-Series Queue → Voting Logic → EWMA → 3단계 Status까지 
    한 번에 처리하는 핵심 클래스"""

    def __init__(self, window_size=7, warning_threshold=0.55, critical_threshold=0.80, 
                 min_conf=0.75, alpha=0.5):
        self.window = deque(maxlen=window_size)
        self.warning_threshold = warning_threshold
        self.critical_threshold = critical_threshold
        self.min_conf = min_conf
        self.alpha = alpha 

    def add_vlm_result(self, raw_json_str: str):
        """VLM 출력 JSON 문자열을 받아 큐에 적재 (process_vlm_json 완전 통합)"""
        data = json.loads(raw_json_str)
        
        # 필수 필드 안전장치
        if 'confidence' not in data:
            data['confidence'] = 0.0
        if 'timestamp' not in data:
            data['timestamp'] = datetime.now().strftime("%H:%M:%S")
        
        conf = float(data.get('confidence', 0.0))
        if conf < self.min_conf:
            data['fire_detected'] = False
            data['smoke_detected'] = False
        
        self.window.append(data)   # 전체 JSON 저장

    def get_decision(self):
        """Voting Logic + EWMA + Status Mapping (E-gle Eye 정확 기준)"""
        if ratio >= 0.92 or (ratio >= self.critical_threshold and ewma >= 0.72): status, alert, level = 'Red', True, 2
        elif ratio >= 0.85 or (ratio >= self.warning_threshold and ewma >= 0.55): status, alert, level = 'Orange', True, 1
        else: status, alert, level = 'Green', False, 0

        # ==================== Voting Logic + EWMA ====================
        fire_weighted_sum = 0.0
        total_weight = 0.0
        confidences = []
        
        for frame in self.window:
            # severity에 따른 가중치
            sev_weight = 1.0 if frame.get('severity') == 'critical' else \
                         0.8 if frame.get('severity') == 'high' else \
                         0.5 if frame.get('severity') == 'medium' else 0.2
            
            w = frame['confidence'] * sev_weight
            
            if frame.get('fire_detected') or frame.get('smoke_detected'):
                fire_weighted_sum += w
            total_weight += w
            confidences.append(frame['confidence'])
        
        ratio = fire_weighted_sum / total_weight if total_weight > 0 else 0.0
        
        # EWMA (alpha=0.35)
        if confidences:
            ewma = confidences[-1]
            alpha = self.alpha
            for c in reversed(confidences[:-1]):
                ewma = alpha * c + (1 - alpha) * ewma
        else:
            ewma = 0.0
        
        # ==================== Status Mapping ====================
        if ratio >= self.critical_threshold and ewma >= 0.75:
            status, alert, level = 'Red', True, 2
        elif ratio >= self.warning_threshold and ewma >= 0.62:
            status, alert, level = 'Orange', True, 1
        else:
            status, alert, level = 'Green', False, 0
        
        return {
            'alert': alert,
            'status': status,
            'level': level,
            'confidence': round(ewma * 100, 1),
            'anomaly_ratio': round(ratio * 100, 1),
            'explanation': f"{len(self.window)}프레임 중 Fire/Smoke 비중 {ratio:.1%} (EWMA {ewma:.3f}) → {status}",
            'latest_json': dict(self.window[-1]) if self.window else {}
        }

print("✅ E-gle Eye SmartTemporalLogic VLM 통합 클래스 로드 완료 (7프레임 / severity 가중 / EWMA 완비)")

✅ E-gle Eye SmartTemporalLogic VLM 통합 클래스 로드 완료 (7프레임 / severity 가중 / EWMA 완비)


In [6]:
class DummyFireSmokeDetector:
    """Azure가 준비될 때까지 사용하는 테스트용 Detector"""
    def __init__(self):
        print("🔥 Dummy Detector 활성화 (실제 Azure 교체 예정)")

    def detect(self, scenario="normal"):
        if scenario == "smoke_rising":
            conf = random.uniform(0.75, 0.98)
            return {'class': 'Smoke', 'confidence': conf}
        elif scenario == "fire":
            conf = random.uniform(0.82, 0.99)
            return {'class': 'Fire', 'confidence': conf}
        else:  # normal
            conf = random.uniform(0.60, 0.85)
            return {'class': 'Normal', 'confidence': conf}

detector = DummyFireSmokeDetector()
logic = SmartTemporalLogic(window_size=5)
print("✅ Dummy Detector & Logic 초기화 완료")

🔥 Dummy Detector 활성화 (실제 Azure 교체 예정)
✅ Dummy Detector & Logic 초기화 완료


# FireAlarm.ipynb 파일 생성 후 코드 수정 후 적용테스트

In [10]:
from collections import deque
import json
from datetime import datetime

class SmartTemporalLogic:
    """E-gle Eye Reliability Validation Layer v1.2
    로직 순서 오류 완전 수정 + 실무 지연 최소화 버전"""

    def __init__(self, window_size=7, warning_threshold=0.55, critical_threshold=0.80, 
                 min_conf=0.62, alpha=0.50, verbose=False):
        self.window = deque(maxlen=window_size)
        self.warning_threshold = warning_threshold
        self.critical_threshold = critical_threshold
        self.min_conf = min_conf
        self.alpha = alpha
        self.verbose = verbose

    def add_vlm_result(self, raw_json_str: str):
        data = json.loads(raw_json_str)
        
        if 'confidence' not in data:
            data['confidence'] = 0.0
        if 'timestamp' not in data:
            data['timestamp'] = datetime.now().strftime("%H:%M:%S")
        
        conf = float(data.get('confidence', 0.0))
        if conf < self.min_conf:
            data['fire_detected'] = False
            data['smoke_detected'] = False
        
        self.window.append(data)
        if self.verbose:
            print(f"[ADD] conf={conf:.2f} → fire/smoke={data.get('fire_detected')}/{data.get('smoke_detected')} (sev={data.get('severity')})")

    def get_decision(self):
        if len(self.window) < 3:
            return {
                'alert': False, 'status': 'Green', 'level': 0,
                'confidence': 0.0, 'anomaly_ratio': 0.0,
                'explanation': '데이터 수집 중...', 'latest_json': {}
            }

        # 1. Voting Logic (가장 먼저 계산)
        fire_weighted_sum = 0.0
        total_weight = 0.0
        confidences = []
        
        for frame in self.window:
            sev_weight = 1.0 if frame.get('severity') == 'critical' else \
                         0.8 if frame.get('severity') == 'high' else \
                         0.5 if frame.get('severity') == 'medium' else 0.2
            w = frame['confidence'] * sev_weight
            
            if frame.get('fire_detected', False) or frame.get('smoke_detected', False):
                fire_weighted_sum += w
            total_weight += w
            confidences.append(frame['confidence'])
        
        ratio = fire_weighted_sum / total_weight if total_weight > 0 else 0.0
        
        # 2. EWMA 계산
        if confidences:
            ewma = confidences[-1]
            alpha = self.alpha
            for c in reversed(confidences[:-1]):
                ewma = alpha * c + (1 - alpha) * ewma
        else:
            ewma = 0.0
        
        # 3. Status Mapping v1.2 (지연 최소화 + ratio 우선 룰)
        if ratio >= 0.92 or (ratio >= self.critical_threshold and ewma >= 0.70):
            status, alert, level = 'Red', True, 2
        elif ratio >= 0.85 or (ratio >= self.warning_threshold and ewma >= 0.56):
            status, alert, level = 'Orange', True, 1
        else:
            status, alert, level = 'Green', False, 0
        
        decision = {
            'alert': alert,
            'status': status,
            'level': level,
            'confidence': round(ewma * 100, 1),
            'anomaly_ratio': round(ratio * 100, 1),
            'explanation': f"{len(self.window)}프레임 중 Fire/Smoke 비중 {ratio:.1%} (EWMA {ewma:.3f}) → {status}",
            'latest_json': dict(self.window[-1]) if self.window else {}
        }
        
        if self.verbose:
            print(f"[DECISION] Ratio={ratio:.3f} | EWMA={ewma:.3f} → {status}")
        
        return decision

# ====================== 개선된 테스트 케이스 ======================
if __name__ == "__main__":
    print("🔥 E-gle Eye SmartTemporalLogic v1.2 테스트 시작 (순서 오류 수정 + Red 도달 용이)\n")
    
    logic = SmartTemporalLogic(verbose=True)
    
    test_cases = [
        '{"fire_detected": false, "smoke_detected": false, "confidence": 0.50, "severity": "low"}',
        '{"fire_detected": true, "smoke_detected": true, "confidence": 0.68, "severity": "medium"}',
        '{"fire_detected": true, "smoke_detected": true, "confidence": 0.75, "severity": "medium"}',
        '{"fire_detected": true, "smoke_detected": true, "confidence": 0.82, "severity": "high"}',
        '{"fire_detected": true, "smoke_detected": true, "confidence": 0.89, "severity": "high"}',
        '{"fire_detected": true, "smoke_detected": true, "confidence": 0.93, "severity": "critical"}',
        '{"fire_detected": true, "smoke_detected": true, "confidence": 0.95, "severity": "critical"}',
        '{"fire_detected": true, "smoke_detected": true, "confidence": 0.97, "severity": "critical"}',
    ]
    
    for i, json_str in enumerate(test_cases, 1):
        logic.add_vlm_result(json_str)
        decision = logic.get_decision()
        print(f"[{i}프레임] {decision['explanation']}")
        print(f"   → Status: {decision['status']} | Alert: {decision['alert']} | EWMA: {decision['confidence']}%")
        if decision['latest_json']:
            print(f"   latest severity: {decision['latest_json'].get('severity')}")
        print()
    
    print("✅ v1.2 테스트 완료! (3~4프레임 Orange, 6~7프레임 Red 목표 달성)")

🔥 E-gle Eye SmartTemporalLogic v1.2 테스트 시작 (순서 오류 수정 + Red 도달 용이)

[ADD] conf=0.50 → fire/smoke=False/False (sev=low)
[1프레임] 데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%

[ADD] conf=0.68 → fire/smoke=True/True (sev=medium)
[2프레임] 데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%

[ADD] conf=0.75 → fire/smoke=True/True (sev=medium)
[DECISION] Ratio=0.877 | EWMA=0.608 → Orange
[3프레임] 3프레임 중 Fire/Smoke 비중 87.7% (EWMA 0.608) → Orange
   → Status: Orange | Alert: True | EWMA: 60.8%
   latest severity: medium

[ADD] conf=0.82 → fire/smoke=True/True (sev=high)
[DECISION] Ratio=0.932 | EWMA=0.616 → Red
[4프레임] 4프레임 중 Fire/Smoke 비중 93.2% (EWMA 0.616) → Red
   → Status: Red | Alert: True | EWMA: 61.6%
   latest severity: high

[ADD] conf=0.89 → fire/smoke=True/True (sev=high)
[DECISION] Ratio=0.954 | EWMA=0.621 → Red
[5프레임] 5프레임 중 Fire/Smoke 비중 95.4% (EWMA 0.621) → Red
   → Status: Red | Alert: True | EWMA: 62.1%
   latest severity: high

[ADD] conf=0.93 → fire/smoke=True/T

# 세번째 테스트 버전

In [11]:
from collections import deque
import json
from datetime import datetime
from pathlib import Path

# ====================== SmartTemporalLogic v1.2 Lock (변경 금지) ======================
class SmartTemporalLogic:
    """E-gle Eye Reliability Validation Layer v1.2
    Time-Series Queue → Voting Logic → EWMA → 3단계 Status까지 처리하는 핵심 클래스
    (이 클래스 내부 로직은 Lock 상태로 유지됩니다)"""

    def __init__(self, window_size=7, warning_threshold=0.55, critical_threshold=0.80, 
                 min_conf=0.62, alpha=0.50, verbose=True):
        # window_size: 최근 몇 프레임을 기억할지 결정 (슬라이딩 윈도우)
        # warning_threshold: Fire/Smoke 비중 기준으로 Orange 시작
        # critical_threshold: Fire/Smoke 비중 기준으로 Red 시작
        # min_conf: VLM 신뢰도 하한선 (이 아래는 무조건 Normal 처리 → 오탐 방지)
        # alpha: EWMA에서 최근 프레임의 가중치 (높을수록 최근 변화에 빠르게 반응)
        # verbose: True면 상세 로그 출력 (테스트/디버깅용)
        self.window = deque(maxlen=window_size)
        self.warning_threshold = warning_threshold
        self.critical_threshold = critical_threshold
        self.min_conf = min_conf
        self.alpha = alpha
        self.verbose = verbose

    def add_vlm_result(self, raw_json_str: str):
        """VLM이 출력한 JSON 문자열을 받아 큐에 저장
        신뢰도가 낮으면 강제로 Normal 처리"""
        data = json.loads(raw_json_str)
        
        # 필수 필드 안전장치
        if 'confidence' not in data:
            data['confidence'] = 0.0
        if 'timestamp' not in data:
            data['timestamp'] = datetime.now().strftime("%H:%M:%S")
        
        conf = float(data.get('confidence', 0.0))
        # min_conf 미만이면 오탐 방지 위해 강제 Normal
        if conf < self.min_conf:
            data['fire_detected'] = False
            data['smoke_detected'] = False
        
        self.window.append(data)   # 전체 JSON을 큐에 저장
        
        if self.verbose:
            print(f"[ADD] conf={conf:.2f} → fire/smoke={data.get('fire_detected')}/{data.get('smoke_detected')} (sev={data.get('severity')})")

    def get_decision(self):
        """Voting Logic + EWMA + Status Mapping 수행
        최종적으로 Green / Orange / Red 결정"""
        if len(self.window) < 3:   # 최소 3프레임 필요
            return {'alert': False, 'status': 'Green', 'level': 0,
                    'confidence': 0.0, 'anomaly_ratio': 0.0,
                    'explanation': '데이터 수집 중...', 'latest_json': {}}

        # ==================== 1. Voting Logic ====================
        fire_weighted_sum = total_weight = 0.0
        confidences = []
        
        for frame in self.window:
            # severity에 따라 가중치 부여 (critical은 가장 강하게 반영)
            sev_weight = 1.0 if frame.get('severity') == 'critical' else \
                         0.8 if frame.get('severity') == 'high' else \
                         0.5 if frame.get('severity') == 'medium' else 0.2
            w = frame['confidence'] * sev_weight
            
            if frame.get('fire_detected', False) or frame.get('smoke_detected', False):
                fire_weighted_sum += w
            total_weight += w
            confidences.append(frame['confidence'])
        
        ratio = fire_weighted_sum / total_weight if total_weight > 0 else 0.0   # Fire/Smoke 전체 비중

        # ==================== 2. EWMA (지수 가중 이동 평균) ====================
        if confidences:
            ewma = confidences[-1]   # 가장 최근 프레임부터 시작
            alpha = self.alpha
            for c in reversed(confidences[:-1]):   # 과거 프레임으로 거꾸로 계산
                ewma = alpha * c + (1 - alpha) * ewma
        else:
            ewma = 0.0

        # ==================== 3. Status Mapping ====================
        if ratio >= 0.92 or (ratio >= self.critical_threshold and ewma >= 0.70):
            status, alert, level = 'Red', True, 2      # 위험
        elif ratio >= 0.85 or (ratio >= self.warning_threshold and ewma >= 0.56):
            status, alert, level = 'Orange', True, 1   # 경계
        else:
            status, alert, level = 'Green', False, 0   # 정상

        decision = {
            'alert': alert,
            'status': status,
            'level': level,
            'confidence': round(ewma * 100, 1),
            'anomaly_ratio': round(ratio * 100, 1),
            'explanation': f"{len(self.window)}프레임 중 Fire/Smoke {ratio:.1%} (EWMA {ewma:.3f}) → {status}",
            'latest_json': dict(self.window[-1]) if self.window else {}
        }
        
        if self.verbose:
            print(f"[DECISION] Ratio={ratio:.3f} | EWMA={ewma:.3f} → {status}")
        
        return decision

# ====================== (나머지 부분: Mock VLM, DataLoader, 메인 테스트는 이전 코드 그대로) ======================
# 아래 부분은 필요에 따라 그대로 사용하세요 (주석 추가 완료)
print("✅ SmartTemporalLogic v1.2 Lock 완료")
print("클래스 변경 없이 이미지 테스트만 진행")

✅ SmartTemporalLogic v1.2 Lock 완료
클래스 변경 없이 이미지 테스트만 진행


In [24]:
# ====================== EgleEyeDataLoader (train image 폴더 읽기) ======================
class EgleEyeDataLoader:
    def __init__(self, root_path: str):
        self.root = Path(root_path)
    
    def get_image_list(self, category: str):
        """fire / smoke / normal 폴더의 이미지 파일 목록 반환"""
        folder = self.root / category
        if not folder.exists():
            print(f"⚠️ 폴더 없음: {folder}")
            return []
        return sorted(list(folder.glob("*.jpg")) + 
                      list(folder.glob("*.png")) + 
                      list(folder.glob("*.jpeg")))

# ====================== Mock VLM (train image 기반 JSON 생성) ======================
def mock_vlm_from_image(image_path: Path, category: str) -> str:
    """실제 이미지에 따라 VLM처럼 JSON 반환 (Lock 유지하면서 confidence 강화)"""
    conf_map = {'normal': 0.30, 'smoke': 0.85, 'fire': 0.95}
    sev_map = {'normal': 'low', 'smoke': 'high', 'fire': 'critical'}
    
    data = {
        "fire_detected": category in ['smoke', 'fire'],
        "smoke_detected": category in ['smoke', 'fire'],
        "confidence": conf_map[category],
        "severity": sev_map[category],
        "fire_location": f"{category} 구역 - {image_path.name}",
        "smoke_color": "black" if category == 'fire' else "gray",
        "possible_cause": ["기계 과열", "전기 과부하"] if category != 'normal' else [],
        "risk_factors": ["가연물 다수", "인명 위험"] if category != 'normal' else [],
        "recommended_action": "즉시 대피 및 119 신고" if category != 'normal' else "정상 모니터링",
        "timestamp": datetime.now().strftime("%H:%M:%S"),
        "image_source": str(image_path)
    }
    return json.dumps(data, ensure_ascii=False)

if __name__ == "__main__":
    ROOT_PATH = r"E:\새 폴더\project pod\train image"   # ← 사용자님 경로 그대로
    
    loader = EgleEyeDataLoader(ROOT_PATH)
    logic = SmartTemporalLogic(verbose=True)
    
    print("🔥 E-gle Eye End-to-End 테스트 시작 (fire/smoke/normal 실제 이미지 사용)\n")
    
    # 1. normal → smoke → fire 순차 테스트 (실제 공장 상황 재현)
    sequence = ['normal', 'normal', 'smoke', 'smoke', 'fire', 'fire', 'fire']
    
    for idx, cat in enumerate(sequence):
        images = loader.get_image_list(cat)
        if not images:
            print(f"❌ {cat} 폴더에 이미지가 없습니다!")
            continue
            
        img = images[idx % len(images)]   # 순환하며 사용
        raw_json = mock_vlm_from_image(img, cat)
        
        print(f"\n=== {idx+1}프레임 : {cat.upper()} 이미지 {img.name} ===")
        logic.add_vlm_result(raw_json)
        decision = logic.get_decision()
        
        print(f"   {decision['explanation']}")
        print(f"   → Status: {decision['status']} | Alert: {decision['alert']} | EWMA: {decision['confidence']}%")
    
    print("\n✅ End-to-End 테스트 완료!")
    print("   normal에서 Green 유지 → smoke에서 Orange → fire에서 Red 빠르게 전환되는지 확인.")

🔥 E-gle Eye End-to-End 테스트 시작 (fire/smoke/normal 실제 이미지 사용)


=== 1프레임 : NORMAL 이미지 01.23432942.1.jpg ===
[ADD] conf=0.30 → fire/smoke=False/False (sev=low)
   데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%

=== 2프레임 : NORMAL 이미지 101745290.3.jpg ===
[ADD] conf=0.30 → fire/smoke=False/False (sev=low)
   데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%

=== 3프레임 : SMOKE 이미지 1238684_1104087_1501.jpg ===
[ADD] conf=0.85 → fire/smoke=True/True (sev=high)
[DECISION] Ratio=0.850 | EWMA=0.438 → Orange
   3프레임 중 Fire/Smoke 85.0% (EWMA 0.438) → Orange
   → Status: Orange | Alert: True | EWMA: 43.8%

=== 4프레임 : SMOKE 이미지 1296627_1489018_2256.jpg ===
[ADD] conf=0.85 → fire/smoke=True/True (sev=high)
[DECISION] Ratio=0.919 | EWMA=0.438 → Orange
   4프레임 중 Fire/Smoke 91.9% (EWMA 0.438) → Orange
   → Status: Orange | Alert: True | EWMA: 43.8%

=== 5프레임 : FIRE 이미지 200463131_700.jpg ===
[ADD] conf=0.95 → fire/smoke=True/True (sev=critical)
[DECISION] Ratio=0.951 | EWMA=0.444 → Red


In [ ]:
from collections import deque
import json
from datetime import datetime
from pathlib import Path

def mock_vlm_from_image(image_path: Path, category: str, frame_idx: int) -> str:
    if category == 'normal':
        conf = 0.30
        sev = 'low'
    elif category == 'smoke':
        conf = 0.82 + (frame_idx % 5) * 0.025   # 0.82 → 0.94 점진 상승
        sev = 'high'
    else:  # fire
        conf = 0.93 + (frame_idx % 5) * 0.012   # 0.93 → 0.99 점진 상승
        sev = 'critical'
    
    data = {
        "fire_detected": category in ['smoke', 'fire'],
        "smoke_detected": category in ['smoke', 'fire'],
        "confidence": round(conf, 2),
        "severity": sev,
        "fire_location": f"{category} 구역 - {image_path.name}",
        "smoke_color": "black" if category == 'fire' else "gray",
        "possible_cause": ["기계 과열", "전기 과부하"] if category != 'normal' else [],
        "risk_factors": ["가연물 다수", "인명 위험"] if category != 'normal' else [],
        "recommended_action": "즉시 대피 및 119 신고" if category != 'normal' else "정상 모니터링",
        "timestamp": datetime.now().strftime("%H:%M:%S"),
        "image_source": str(image_path)
    }
    return json.dumps(data, ensure_ascii=False)

# ====================== train image 전용 DataLoader ======================
class TrainImageLoader:
    def __init__(self, root_path: str):
        self.root = Path(root_path)
    
    def get_images(self, category: str):
        folder = self.root / category
        if not folder.exists():
            print(f"⚠️ 폴더 없음: {folder}")
            return []
        return sorted(list(folder.glob("*.jpg")) + list(folder.glob("*.png")) + list(folder.glob("*.jpeg")))
    
# ====================== 메인 테스트 ======================
if __name__ == "__main__":
    ROOT_PATH = r"E:\새 폴더\project pod\train image"
    
    loader = TrainImageLoader(ROOT_PATH)
    logic = SmartTemporalLogic(verbose=True)
    
    print("🔥 E-gle Eye Lock 유지 테스트 시작 (Mock VLM confidence 강화)\n")
    
    sequence = ['normal']*5 + ['smoke']*5 + ['fire']*5   # 총 15프레임 현실 시뮬
    
    for idx, cat in enumerate(sequence):
        images = loader.get_images(cat)
        if not images:
            print(f"❌ {cat} 폴더에 이미지가 없습니다!")
            continue
        img = images[idx % len(images)]
        
        raw_json = mock_vlm_from_image(img, cat, idx)
        
        print(f"\n===# 🔥 Fire Alarm System - Reliability Validation Layer 테스트
### Time-Series Queue → Voting Logic → Action Trigger 완전 동작 확인용 {idx+1}프레임 : {cat.upper()} - {img.name} ===")
        logic.add_vlm_result(raw_json)
        decision = logic.get_decision()
        
        print(f"   {decision['explanation']}")
        print(f"   → Status: {decision['status']} | Alert: {decision['alert']} | EWMA: {decision['confidence']}%")
    
    print("\n✅ Lock 유지 테스트 완료!")
    print("   EWMA가 정상적으로 상승하는지, Orange/Red 전환이 빨라졌는지 확인.")

🔥 E-gle Eye Lock 유지 테스트 시작 (Mock VLM confidence 강화)


=== 1프레임 : NORMAL - 01.23432942.1.jpg ===
[ADD] conf=0.30 → fire/smoke=False/False (sev=low)
   데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%

=== 2프레임 : NORMAL - 101745290.3.jpg ===
[ADD] conf=0.30 → fire/smoke=False/False (sev=low)
   데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%

=== 3프레임 : NORMAL - 1079900_845870_327.jpg ===
[ADD] conf=0.30 → fire/smoke=False/False (sev=low)
[DECISION] Ratio=0.000 | EWMA=0.300 → Green
   3프레임 중 Fire/Smoke 0.0% (EWMA 0.300) → Green
   → Status: Green | Alert: False | EWMA: 30.0%

=== 4프레임 : NORMAL - 116364906.1.jpg ===
[ADD] conf=0.30 → fire/smoke=False/False (sev=low)
[DECISION] Ratio=0.000 | EWMA=0.300 → Green
   4프레임 중 Fire/Smoke 0.0% (EWMA 0.300) → Green
   → Status: Green | Alert: False | EWMA: 30.0%

=== 5프레임 : NORMAL - 142_dsculture_20121005_3.jpg ===
[ADD] conf=0.30 → fire/smoke=False/False (sev=low)
[DECISION] Ratio=0.000 | EWMA=0.300 → Green
   5프레임 중 Fire/Smok